<a href="https://colab.research.google.com/github/MahiKhan5360/Deep-Learning-models-for-Biomedical-Image-Processing/blob/main/FCN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn

class ConvBlock(nn.Modules):
  def __init__(self, in_chaneels, out_channels, num_layers):
    super(ConvBlock, self).__init__()
    layers =  [nn.Conv2d(in_channels, out_channels, kernel_size = 3 , padding = 1, bias = False), nn.BatchNorm2d(out_channels), nn.ReLU(inplace= True)]
    for _ in range (num_layers-1):
      Layers += [nn.Conv2d(out_channels, out_channels, Kernel_size = 3, padding =1 , bias = False), nn.BatchNorm2d(out_channels), nn.RelU(inplace = True)]
    self.conv = nn. Sequential(*layers)

   def forward(self,x):
    return self.conv(x)

class FCN8s(nn.Module):

   def __init__(self, n_class, pretrained = True):
    super(FCN8S, self).__init__()
    #VGG16 Backbone

    self.Conv1 = ConvBlock(3, 64, 2)
    self.Pool1 = nn.MaxPool2d(kernel_size = 2, stride = 2)
    self.Conv2 = ConvBlock(64, 128, 2)
    self.Pool2 = nn.MaxPool2d(kernel_size = 2, stride = 2)
    self.Conv3 = ConvBlock(128, 256, 3)
    self.Pool3 = nn.Maxpool2d(kernel_size = 2, stride = 2)
    self.Conv4 = ConvBlock(256, 512, 3)
    self.Pool4 = nn.Maxpool2d(kernel_size = 2, stride = 2)
    self.Conv5 = ConvBlock(512, 512, 3)
    self.Pool5 = nn.Maxpool2d(kernel_size = 2, stride = 2)

    self.Conv6 = nn.sequential(
        nn.Conv2d(512, 4096, kernel_size = 7, padding = 0)
        nn.ReLu(inplace = True)
        nn.Dropout2d(0.5)
    )

    self.Conv7 = nn.sequential(
        nn.Conv2d(4096, 4096, Kernel_size = 1, padding = 0)
        nn.ReLu(inplace = True)
        nn.Dropout2d(0.5)
    )
    self.score_fr = nn.Conv2d(4096, n_class, kernel_size= 1)

    #skip connection and upsampling
    self.upscore2 = nn.ConvTranspose2d(n_class, n_class, kernel_size = 4, stride = 2, padding = 1) #2X up
    self.score_pool4 = nn.Conv2d( 512, n_class, kernel_size = 1)
    self.upscore_pool3 = nn.ConvTranspose2d(n_class, n_class, kernel_size = 4, stride = 2, padding = 1) #2X up
    self.score_pool3 = nn.Conv2d(256, n_class, kernel_size = 1)
    self.upscore8 = nn.ConvTranspose2d(n_class, n_class, kernel_size = 16, stride = 8, padding = 4) #8X up

    # Load pre-trained VGG16 weights if requested
        if pretrained:
            vgg = models.vgg16(pretrained=True).features
            self.load_vgg_weights(vgg)

  def load_vgg_weights(self, vgg):
        """Load pre-trained VGG16 weights into conv1 to conv5."""
        vgg_layers = list(vgg)
        for block, target in [
            (self.conv1, vgg_layers[:4]),   # 2 conv + ReLU
            (self.conv2, vgg_layers[5:9]),  # 2 conv + ReLU
            (self.conv3, vgg_layers[10:16]), # 3 conv + ReLU
            (self.conv4, vgg_layers[17:23]), # 3 conv + ReLU
            (self.conv5, vgg_layers[24:30])  # 3 conv + ReLU
        ]:

            conv_idx = 0
            for i, layer in enumerate(target):
                if isinstance(layer, nn.Conv2d):
                    block.conv[conv_idx * 3].weight.data.copy_(layer.weight.data)
                    conv_idx += 1

    def forward(self, x):
        """Forward pass with skip connections and upsampling."""
        # VGG backbone
        x1 = self.conv1(x)
        p1 = self.pool1(x1)
        x2 = self.conv2(p1)
        p2 = self.pool2(x2)
        x3 = self.conv3(p2)
        p3 = self.pool3(x3)  # Save pool3 for skip connection
        x4 = self.conv4(p3)
        p4 = self.pool4(x4)  # Save pool4 for skip connection
        x5 = self.conv5(p4)
        p5 = self.pool5(x5)  # Save pool5

        # Fully convolutional layers
        x6 = self.conv6(p5)
        x7 = self.conv7(x6)
        score_fr = self.score_fr(x7)

        # Skip connections and upsampling
        up2 = self.upscore2(score_fr)  # 2x upsample
        score_pool4 = self.score_pool4(p4)
        fuse4 = score_pool4 + up2  # Skip connection with pool4
        up4 = self.upscore_pool4(fuse4)  # 2x upsample
        score_pool3 = self.score_pool3(p3)
        fuse3 = score_pool3 + up4  # Skip connection with pool3
        out = self.upscore8(fuse3)  # 8x upsample to original size
        return out
